**Curso de Inteligencia Artificial y Aprendizaje Profundo**


# Introducción al aprendizaje reforzado

## Q-Learning con redes neuronales, algoritmo DQN.

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [Ecuación de Bellman en DQN](#Ecuación-de-Bellman-en-DQN)
* [Experience replay](#Experience-replay)
* [Freezing the target network](#Freezing-the-target-network)
* [Algoritmo DQN](#Algoritmo-DQN)
* [Ejemplo ambiente CartPole-v0 en GymImporta módulos ](#Ejemplo-ambiente-CartPole-v0-en-Gym-Importa-módulos )
* [Crea la clase DQNAgent](#Crea-la-clase-DQNAgent)
* [Algoritmo DQN](#Algoritmo-DQN)
* [Clase-DDQNA](#Clase-DDQNA)
* [Video luego de entrenado el agente](#Video-luego-de-entrenado-el-agente)


## Referencias

1. Adaptado de Rowel Atienza, [Advance Deep Learning with Tensorflow 2 and Keras](https://www.amazon.com/-/es/Rowel-Atienza-ebook/dp/B0851D5YQQ), Pack,2020.
2. Sutton, R. S., & Barto, A. G. (2018). [Reinforcement learning: An introduction. MIT press](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf).
3. [Ejecutar en Colab](https://colab.research.google.com/drive/1ExE__T9e2dMDKbxrJfgp8jP0So8umC-A#sandboxMode=true&scrollTo=2XelFhSJGWGX)

## Introducción

En la lección del algoritmo Q-Learning, este  funciona muy bien cuando el entorno es simple y la función $Q(s,a)$ se puede representar como una tabla o matriz de valores. 

Pero cuando hay miles de millones de estados diferentes y cientos de acciones distintas, la tabla se vuelve enorme, y no es viable su utilización. 

Por ello, Mnih et al. inventaron el algoritmo [Deep Q-Network o DQN.](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf)  

Este algoritmo combina el algoritmo *Q-learning* con redes neuronales profundas (Deep Neural Networks). 

Como es sabido en el campo de la IA, las redes neuronales son una fantástica manera de aproximar funciones no lineales. 

Por lo tanto, este algoritmo usa una red neuronal para aproximar la función *Q*, evitando así utilizar una tabla para representar la misma. 

En realidad, utiliza dos redes neuronales para estabilizar el proceso de aprendizaje. 

1. La primera, la *red neuronal principal* (main Neural Network), representada por los parámetros $\theta$, se utiliza para estimar los valores-Q del estado s y acción a actuales: $Q(s, a;\theta)$. 
2. La segunda, la *red neuronal objetivo* (target Neural Network), parametrizada por $\theta^{'}$, tendrá la misma arquitectura que la red principal pero se usará para aproximar los *valores-Q* del siguiente estado $s'$ y la siguiente acción $a'$. 

El aprendizaje ocurre en la red principal y no en la objetivo.

La red objetivo se congela (sus parámetros no se cambian) durante varias iteraciones (normalmente alrededor de 10000), y después los parámetros de la red principal se copian a la red objetivo, transmitiendo así el aprendizaje de una a otra, haciendo que las estimaciones calculadas por la red objetivo sean más precisas.

La siguiente imagen compara los procesos Q-learning y DN-learning.

<figure>
<center>
<img src="../Imagenes/Q-Network.png" width="600" height="500" align="center"/>
    <figcaption>
<p style="text-align:center">Comparación  Q-learning v.s. DQ Learning</p>
</figcaption>
</figure>



Fuente: [reinforcement-learning-deep-q-networks](https://blogs.oracle.com/datascience/reinforcement-learning-deep-q-networks)

Los datos requerido para entrenar la Q-network provienen de  la experiencia del agente: $(s_0 a_0 r_1 s_1, s_1 a_1 r_2  s_2,\ldots, s_{T-1} a_{T-1} r_T s_T)$. Cada muestra de entrnamiento es una unidad de experiencia, $s_t a_t r_{t+1} s_{t+1}$.

En el tiempo $t$ se tiene el estado $s=s_t$.  La acción $a=a_t$ es determinada usando el algoritmo Q-learning similar a las lecciones previas:

$$
\pi(s) = \begin{cases} sample(a) &\text{ random } < \epsilon \\
\arg \max_a{ Q(s,a)} &\text{ en otro caso }\end{cases}
$$

## Ecuación de Bellman en DQN


La ecuación de Bellman tiene esta forma ahora. 

$$
\large
Q(s,a; \theta) = r + \lambda \max_{a^{'}}  Q(s^{'},a^{'}; \theta^{'}).
$$

Para poder entrenar una red neuronal, necesitamos una función de pérdida o coste (loss or cost function), la cual definimos como el cuadrado de la diferencia entre ambos lados de la ecuación de Bellman:

$$
\large
L(\theta)= \mathbb{E}[  r + \lambda \max_{a^{'}}  Q(s^{'},a^{'}; \theta^{'})- Q(s,a; \theta)]^2.
$$


Esta será la función que minimizaremos usando el algoritmo de descenso de gradiente (gradient descent), el cuál se ejecuta automáticamente si usamos una librería de diferenciación automática con redes neuronales, como TensorFlow.

Desde el punto de vista del algoritmo, en este caso se usa la $Q$-network para predecir el valor $Q$ de cada posible siguiente acción, dado el estado siguiente y escogiendo el máximo entre ellos.

En este caso, que en el estado terminal se tiene que $\max_{a^{'}}  Q(s^{'},a^{'})=0$.

## Experience replay


Por lo general el entrenamiento de la Q-network es inestable. Hay dos causas de la instabilidad

1. Alta correlación entre las muestras.
2. El target no es estacionario.

Para resolver el problema de la alta correlación, los datos de entrenamiento son seleccionados aleatorios de un buffer que creamos para tal fín. 

Este proceso se conoce como exprience replay (experiencia de repetición.

##  Freezing the target network


EL problema de no estacionariedad de la red objetivo (target) es resuelto congelando los pesos de la red objetivo durante $C$ pasos de  entrenamiento.


Es decir, en realidad esta es la razón de tener dos redes idénticas. Los parámetros de la Q-network target son copiados desde la Q-network principal cada $C$ pasos de entrenamientos.

## Algoritmo DQN 

- Inicializar la replay memory *D* con cpacidad *N*.
- Inicializar la función de valor Q con pesos aleatorios $\theta$
- Inicializar la función acción-valor target $Q_{\text{target}}$ con pesos con pesos $\theta^{-}= \theta$
- Definir la rata de exploración inicial $\epsilon$ y el factor de descuento, $\gamma$.

1. Para $\text{ episodio } = 1,\ldots, M$, do:
2.        Dado el estado incial *s*
3.        Para step =1,...T do:
4.            Escoja la acción 
$$
a = \begin{cases} sample(a) &\text{ random } < \epsilon \\
\arg \max_a{ Q(s,a)} &\text{ en otro caso }\end{cases}
$$
5. Ejecuta la acción *a* observe la recompensa *r* y el siguiente estado *s'*
6. Almacene la transición $(s,a,r,s')$ en D
7. Actualice el estado $s=s'$
8. // experience replay
9.Tome un mini-lote de muestra de experiencias de episodios $(s_j,a_j, r_{j+1}, s_{j+1})$ desde el buffer *D*.
10.

$$
Q_{\text{max}} = \begin{cases} r_{j+1} &\text{ si el episodio termina en } j+1\\
r_{j+1} + \lambda \max_{a_{j+1}} Q_{\text{target}}(s_{j+1},a_{j+1};\theta ^{-}) &\text{ en otro caso }\\
\end{cases}
$$
11. Ejecuta un paso de gradiente descendiente para la función de pédida $\letf(Q_{\text{max}}- Q(s_j,a_j;\theta) \right)^2$ con respecto a los parámetros
12. // actualización periódica de la red target
13. Cada $C$ pasos, $Q_{\text{target}}=Q$, es decir $\theta^{-} = \theta$
14. fin
15. fin


## Ejemplo ambiente CartPole-v0 en GymImporta módulos 


Aquí tenemos el entorno (environment) conocido como CartPole. Se ha utilizado la librería *OpenAI Gym* para visualizar y ejecutar este entorno. En este entorno, el objetivo es mover el carro hacia la derecha y la izquierda, con el objetivo de equilibrar el palo. Y si el palo se tuerce más de 15 grados respecto al eje vertical, el episodio terminará y volveremos a empezar.

<figure>
<center>
<img src="../Imagenes/Ejemplo_polea.gif" width="400" height="300" align="center"/>
    <figcaption>
<p style="text-align:center">Ejemplo de la polea. Original en Open AI</p>
</figcaption>
</figure>

Fuente: [Introducción al aprendizaje por refuerzo](https://medium.com/@markelsanz14/introducci%C3%B3n-al-aprendizaje-por-refuerzo-parte-3-q-learning-con-redes-neuronales-algoritmo-dqn-bfe02b37017f)

## El código

## Instalar depencendias para renderizar OpenAI Gym Environment

In [ ]:
#!apt-get update
#!pip install pyglet
#!pip install gym pyvirtualdisplay
#!pip install xvfbwrapper
#!apt-get install -y xvfb python-opengl ffmpeg
#!pip install tensorflow==2.1.* 

In [ ]:
Mejor correr en Colab

## Importa módulos 


In [ ]:
"""Trains a DQN/DDQN to solve CartPole-v0 problem


"""

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from collections import deque
import numpy as np
import random
import argparse
import gym
from gym import wrappers, logger

## Crea la clase DQNAgent

In [ ]:
class DQNAgent:
    def __init__(self,
                 state_space, 
                 action_space, 
                 episodes=500):
        """DQN Agent on CartPole-v0 environment

        Arguments:
            state_space (tensor): state space
            action_space (tensor): action space
            episodes (int): number of episodes to train
        """
        self.action_space = action_space

        # experience buffer
        self.memory = []

        # discount rate
        self.gamma = 0.9

        # initially 90% exploration, 10% exploitation
        self.epsilon = 1.0
        # iteratively applying decay til 
        # 10% exploration/90% exploitation
        self.epsilon_min = 0.1
        self.epsilon_decay = self.epsilon_min / self.epsilon
        self.epsilon_decay = self.epsilon_decay ** \
                             (1. / float(episodes))

        # Q Network weights filename
        self.weights_file = 'dqn_cartpole.h5'
        # Q Network for training
        n_inputs = state_space.shape[0]
        n_outputs = action_space.n
        self.q_model = self.build_model(n_inputs, n_outputs)
        self.q_model.compile(loss='mse', optimizer=Adam())
        # target Q Network
        self.target_q_model = self.build_model(n_inputs, n_outputs)
        # copy Q Network params to target Q Network
        self.update_weights()

        self.replay_counter = 0

    
    def build_model(self, n_inputs, n_outputs):
        """Q Network is 256-256-256 MLP

        Arguments:
            n_inputs (int): input dim
            n_outputs (int): output dim

        Return:
            q_model (Model): DQN
        """
        inputs = Input(shape=(n_inputs, ), name='state')
        x = Dense(256, activation='relu')(inputs)
        x = Dense(256, activation='relu')(x)
        x = Dense(256, activation='relu')(x)
        x = Dense(n_outputs,
                  activation='linear', 
                  name='action')(x)
        q_model = Model(inputs, x)
        q_model.summary()
        return q_model


    def save_weights(self):
        """save Q Network params to a file"""
        self.q_model.save_weights(self.weights_file)


    def update_weights(self):
        """copy trained Q Network params to target Q Network"""
        self.target_q_model.set_weights(self.q_model.get_weights())


    def act(self, state):
        """eps-greedy policy
        Return:
            action (tensor): action to execute
        """
        if np.random.rand() < self.epsilon:
            # explore - do random action
            return self.action_space.sample()

        # exploit
        q_values = self.q_model.predict(state)
        # select the action with max Q-value
        action = np.argmax(q_values[0])
        return action


    def remember(self, state, action, reward, next_state, done):
        """store experiences in the replay buffer
        Arguments:
            state (tensor): env state
            action (tensor): agent action
            reward (float): reward received after executing
                action on state
            next_state (tensor): next state
        """
        item = (state, action, reward, next_state, done)
        self.memory.append(item)


    def get_target_q_value(self, next_state, reward):
        """compute Q_max
           Use of target Q Network solves the 
            non-stationarity problem
        Arguments:
            reward (float): reward received after executing
                action on state
            next_state (tensor): next state
        Return:
            q_value (float): max Q-value computed
        """
        # max Q value among next state's actions
        # DQN chooses the max Q value among next actions
        # selection and evaluation of action is 
        # on the target Q Network
        # Q_max = max_a' Q_target(s', a')
        q_value = np.amax(\
                     self.target_q_model.predict(next_state)[0])

        # Q_max = reward + gamma * Q_max
        q_value *= self.gamma
        q_value += reward
        return q_value


    def replay(self, batch_size):
        """experience replay addresses the correlation issue 
            between samples
        Arguments:
            batch_size (int): replay buffer batch 
                sample size
        """
        # sars = state, action, reward, state' (next_state)
        sars_batch = random.sample(self.memory, batch_size)
        state_batch, q_values_batch = [], []

        # fixme: for speedup, this could be done on the tensor level
        # but easier to understand using a loop
        for state, action, reward, next_state, done in sars_batch:
            # policy prediction for a given state
            q_values = self.q_model.predict(state)
            
            # get Q_max
            q_value = self.get_target_q_value(next_state, reward)

            # correction on the Q value for the action used
            q_values[0][action] = reward if done else q_value

            # collect batch state-q_value mapping
            state_batch.append(state[0])
            q_values_batch.append(q_values[0])

        # train the Q-network
        self.q_model.fit(np.array(state_batch),
                         np.array(q_values_batch),
                         batch_size=batch_size,
                         epochs=1,
                         verbose=0)

        # update exploration-exploitation probability
        self.update_epsilon()

        # copy new params on old target after 
        # every 10 training updates
        if self.replay_counter % 10 == 0:
            self.update_weights()

        self.replay_counter += 1

    
    def update_epsilon(self):
        """decrease the exploration, increase exploitation"""
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        

## Clase DDQNA


In [ ]:
class DDQNAgent(DQNAgent):
    def __init__(self,
                 state_space, 
                 action_space, 
                 episodes=500):
        super().__init__(state_space, 
                         action_space, 
                         episodes)
        """DDQN Agent on CartPole-v0 environment

        Arguments:
            state_space (tensor): state space
            action_space (tensor): action space
            episodes (int): number of episodes to train
        """

        # Q Network weights filename
        self.weights_file = 'ddqn_cartpole.h5'
        print("-------------DDQN------------")

    def get_target_q_value(self, next_state, reward):
        """compute Q_max
           Use of target Q Network solves the 
            non-stationarity problem
        Arguments:
            reward (float): reward received after executing
                action on state
            next_state (tensor): next state
        Returns:
            q_value (float): max Q-value computed
        """
        # max Q value among next state's actions
        # DDQN
        # current Q Network selects the action
        # a'_max = argmax_a' Q(s', a')
        action = np.argmax(self.q_model.predict(next_state)[0])
        # target Q Network evaluates the action
        # Q_max = Q_target(s', a'_max)
        q_value = self.target_q_model.predict(\
                                      next_state)[0][action]

        # Q_max = reward + gamma * Q_max
        q_value *= self.gamma
        q_value += reward
        return q_value

In [ ]:
# Configuración
env_id = 'CartPole-v0'
outdir = "../Datos/dqn-%s" % env_id
no_render = True
    
# the number of trials without falling over
win_trials = 100

# the CartPole-v0 is considered solved if 
# for 100 consecutive trials, he cart pole has not 
# fallen over and it has achieved an average 
# reward of 195.0 
# a reward of +1 is provided for every timestep 
# the pole remains upright
win_reward = { 'CartPole-v0' : 195.0 }

# stores the reward per episode
scores = deque(maxlen=win_trials)
logger.setLevel(logger.ERROR)
    
# crea el ambiente desde gym
env = gym.make(env_id)


if no_render:
    env = wrappers.Monitor(env,
                           directory=outdir,
                            video_callable=False,
                            force=True)
else:
    env = wrappers.Monitor(env, directory=outdir, force=True)
env.seed(0)

# instantiate the DQN agent
agent = DQNAgent(env.observation_space, env.action_space)

# should be solved in this number of episodes
episode_count = 3000
 state_size = env.observation_space.shape[0]
batch_size = 64

# by default, CartPole-v0 has max episode steps = 200
# you can use this to experiment beyond 200
# env._max_episode_steps = 4000

# Q-Learning sampling and fitting
for episode in range(episode_count):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    total_reward = 0
    while not done:
        # in CartPole-v0, action=0 is left and action=1 is right
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
         # in CartPole-v0:
        # state = [pos, vel, theta, angular speed]
        next_state = np.reshape(next_state, [1, state_size])
        # store every experience unit in replay buffer
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward


    # call experience relay
    if len(agent.memory) >= batch_size:
         agent.replay(batch_size)
    
    scores.append(total_reward)
    mean_score = np.mean(scores)
    if mean_score >= win_reward[args.env_id] \
            and episode >= win_trials:
        print("Solved in episode %d: \
                   Mean survival = %0.2lf in %d episodes"
                  % (episode, mean_score, win_trials))
        print("Epsilon: ", agent.epsilon)
            agent.save_weights()
        break
    if (episode + 1) % win_trials == 0:
           print("Episode %d: Mean survival = \
                   %0.2lf in %d episodes" %
                  ((episode + 1), mean_score, win_trials))

    # close the env and write monitor result info to disk
env.close() 

## Video luego de entrenado el agente

<figure>
<center>
<img src="../Imagenes/Ejemplo_polea_entrenado.gif" width="400" height="300" align="center"/>
    <figcaption>
<p style="text-align:center">Ejemplo de la polea despues de entrenar al agente. Original en Open AI</p>
</figcaption>
</figure>

Fuente: [Introducción al aprendizaje por refuerzo](https://medium.com/@markelsanz14/introducci%C3%B3n-al-aprendizaje-por-refuerzo-parte-3-q-learning-con-redes-neuronales-algoritmo-dqn-bfe02b37017f)